In [ ]:
%matplotlib inline
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import warnings; warnings.simplefilter('ignore')

In [ ]:
f = np.loadtxt('G011266.txt', dtype = 'str', delimiter = ',', skiprows = 10)

IDs = f[:,0]
print(len(IDs))

proposal = f[:,4]

In [ ]:
ID_list = []

#can adjust this for selecting targets unique to Josh's proposal
#use: if len(p) == 7:
for n, p in zip(IDs, proposal):
    if len(p)>0:
        ID_list.append(n)

order = np.sort(ID_list)

print(len(order))

is_unique = []

for i in order:
    if i not in is_unique:
        is_unique.append(i)

#SHORTENED FOR TESTING PURPOSES
is_unique = is_unique[:30]

print(len(is_unique))

for i in is_unique:

    print(i)
    
    try:

        tpf = lk.search_targetpixelfile('TIC {}'.format(i), sector=5).download()
        
        tpf.plot(aperture_mask=tpf.pipeline_mask)
        lc = tpf.to_lightcurve(aperture_mask=tpf.pipeline_mask)

        # Create a median image of the source over time
        median_image = np.nanmedian(tpf.flux, axis=0)

        # Select pixels which are brighter than the 85th percentile of the median image
        aperture_mask = median_image > np.nanpercentile(median_image, 85)

        # Plot that aperture
        tpf.plot(aperture_mask=aperture_mask)
        lc2 = tpf.to_lightcurve(aperture_mask=aperture_mask)

        clipped_lc = lc.remove_outliers(sigma=6)
        clipped_lc2 = lc2.remove_outliers(sigma=6)
        
        ax = clipped_lc.scatter()
        ax = clipped_lc2.scatter()

        #tpf.interact()

    except:
        pass
    
# Stops plotting at 117748478 every time